In [2]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [3]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-06-10 14:51:47--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 104.26.3.33, 172.67.70.149, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip’

book-crossings.zip  100%[===================>]  24.88M  68.7MB/s    in 0.4s    

2024-06-10 14:51:48 (68.7 MB/s) - ‘book-crossings.zip’ saved [26085508/26085508]

Archive:  book-crossings.zip
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [4]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [5]:
#merge df_books and df_ratings
df = df_ratings.merge(df_books, on='isbn',how='left')

In [6]:
df.head()

,user,isbn,rating,title,author
0,276725,034545104X,0.0,Flesh Tones: A Novel,M. J. Rose
1,276726,0155061224,5.0,Rites of Passage,Judith Rae
2,276727,0446520802,0.0,The Notebook,Nicholas Sparks
3,276729,052165615X,3.0,Help!: Level 1,Philip Prowse
4,276729,0521795028,6.0,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather


In [7]:
df.shape

(1149780, 5)

In [8]:
#get users who left more than 200 reviews
#get books with more than 100
users= df['user'].value_counts()
users= users[users>=200].index
isbn= df['isbn'].value_counts()
isbn= isbn[isbn>=100].index

In [9]:
#make a new data frame
df = df.loc[(df['user'].isin(users.values)) & (df['isbn'].isin(isbn.values))]

In [10]:
#drop a duplicate reviews
df.drop_duplicates(['title','user'])

,user,isbn,rating,title,author
1456,277427,002542730X,10.0,Politically Correct Bedtime Stories: Modern Ta...,James Finn Garner
1469,277427,0060930535,0.0,The Poisonwood Bible: A Novel,Barbara Kingsolver
1471,277427,0060934417,0.0,Bel Canto: A Novel,Ann Patchett
1474,277427,0061009059,9.0,One for the Money (Stephanie Plum Novels (Pape...,Janet Evanovich
1484,277427,0140067477,0.0,The Tao of Pooh,Benjamin Hoff
...,...,...,...,...,...
1147304,275970,0804111359,0.0,Secret History,DONNA TARTT
1147436,275970,140003065X,0.0,A Fine Balance,Rohinton Mistry
1147439,275970,1400031346,0.0,The No. 1 Ladies' Detective Agency,Alexander McCall Smith
1147440,275970,1400031354,0.0,Tears of the Giraffe (No.1 Ladies Detective Ag...,Alexander McCall Smith


In [11]:
df.isna().sum()

user        0
isbn        0
rating      0
title     264
author    264
dtype: int64

In [12]:
#dropping isbns without title and author
df= df.dropna(how='any')

In [13]:
df.shape

(49517, 5)

In [21]:
#drop duplicate titles keeping the first
df.drop_duplicates(subset=['title'], keep='first', inplace=True)

#create a pivot table
df_pivot = df.pivot(index = 'title', columns = 'user', values = 'rating').fillna(0)
df_pivot.head(3)

user,254,2276,2766,2977,3363,4017,6242,6251,6323,6543,...,13273,13552,15408,16106,16795,21014,277427,277478,277639,278418
title,,,,,,,,,,,,,,,,,,,,,
1984,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1st to Die: A Novel,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2nd Chance,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [22]:
#create a matrix
df_matrix = csr_matrix(df_pivot.values)

In [23]:
#creating and training the model
nn = NearestNeighbors(metric = 'cosine')
nn.fit(df_matrix)

NearestNeighbors(metric='cosine')

In [26]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
    #create a list to put our model outputs into
    recommended_books = [book,[]]

    #pass through our book
    distance, book_info = nn.kneighbors([df_pivot.loc[book]],6, return_distance=True)

    #gather the text and distance & reverse the order
    recom_book_info = df_pivot.iloc[np.flip(book_info[0])[:-1]].index.to_list()
    recom_distance = list(np.flip(distance[0])[:-1])

    #for each value in our two variable append to our empty book list above
    for r in zip(recom_book_info, recom_distance):
        recommended_books[1].append(list(r))

    return recommended_books

In [27]:
#test function
get_recommends('The Catcher in the Rye')

['The Catcher in the Rye',
 [['The Beach House', 1.0],
  ['The Bluest Eye', 1.0],
  ['The Big Bad Wolf: A Novel', 1.0],
  ['The Blind Assassin', 1.0],
  ['The Bean Trees', 1.0]]]

In [28]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [['The Beach House', 1.0], ['The Bluest Eye', 1.0], ['The Big Bad Wolf: A Novel', 1.0], ['The Blind Assassin', 1.0], ['The Bean Trees', 1.0]]]
You haven't passed yet. Keep trying!
